<a href="https://colab.research.google.com/github/oluwafemidiakhoa/MLprject/blob/main/AISYTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install openai fastapi uvicorn nest-asyncio gradio
!pip install transformers pandas
!pip install python-dotenv
!pip install gradio

In [7]:
import gradio as gr
import pandas as pd
import numpy as np
from io import StringIO
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline
from openai import OpenAI
import os
from google.colab import userdata
from dotenv import load_dotenv
load_dotenv()

# Set environment variables for OpenAI and Kaggle credentials
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# Initialize OpenAI Client
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Utility Functions for EDA
def read_csv(file):
    """Reads a CSV file and returns a DataFrame."""
    try:
        if isinstance(file, str):
            return pd.read_csv(file)
        else:
            data = file.read().decode("utf-8")
            return pd.read_csv(StringIO(data))
    except Exception as e:
        raise ValueError(f"Error reading CSV file: {e}")

def generate_summary(df):
    """Generate a statistical summary of the dataset."""
    try:
        summary = df.describe(include="all").transpose()
        summary["Missing (%)"] = df.isnull().mean() * 100
        return summary
    except Exception as e:
        return f"Error generating summary: {e}"

def plot_heatmap(df):
    """Generate a heatmap for the correlation matrix."""
    plt.figure(figsize=(10, 8))
    numeric_df = df.select_dtypes(include=[np.number])
    if numeric_df.empty:
        return None
    sns.heatmap(numeric_df.corr(), annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Correlation Heatmap")
    plt.tight_layout()
    plt.savefig("correlation_heatmap.png")
    plt.close()
    return "correlation_heatmap.png"

def plot_histograms(df):
    """Generate histograms for numerical features."""
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) == 0:
        return None
    fig, axes = plt.subplots(len(numeric_cols), 1, figsize=(10, len(numeric_cols) * 5))
    if len(numeric_cols) == 1:
        axes = [axes]
    for ax, col in zip(axes, numeric_cols):
        sns.histplot(df[col], kde=True, ax=ax, color="blue")
        ax.set_title(f"Histogram of {col}")
    plt.tight_layout()
    plt.savefig("numeric_histograms.png")
    plt.close()
    return "numeric_histograms.png"

def plot_boxplots(df):
    """Generate boxplots for numerical features."""
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if numeric_cols.empty:
        return None
    fig, axes = plt.subplots(len(numeric_cols), 1, figsize=(10, len(numeric_cols) * 5))
    if len(numeric_cols) == 1:
        axes = [axes]
    for ax, col in zip(axes, numeric_cols):
        sns.boxplot(data=df, y=col, ax=ax, color="orange")
        ax.set_title(f"Boxplot of {col}")
    plt.tight_layout()
    plt.savefig("numeric_boxplots.png")
    plt.close()
    return "numeric_boxplots.png"

def comprehensive_eda(file):
    """Perform comprehensive exploratory data analysis."""
    try:
        df = read_csv(file)

        if df.empty:
            return "Error: Uploaded CSV is empty.", None, None, None, None, "No data to analyze."

        # Generate Summary
        summary = generate_summary(df)
        summary_html = summary.style.background_gradient(cmap="Blues").to_html()

        # Generate Visualizations
        heatmap = plot_heatmap(df)
        histograms = plot_histograms(df)
        boxplots = plot_boxplots(df)

        return summary_html, heatmap, histograms, boxplots, "EDA Complete!"
    except Exception as e:
        return f"Error during EDA: {e}", None, None, None, "EDA Failed."

# AI Text Generation
def gpt_generate_text(prompt, creativity=0.7, max_tokens=1500):
    """Generate high-quality text using GPT-4."""
    try:
        response = openai_client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=creativity,
            max_tokens=max_tokens
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Hugging Face Summarization
def summarize_text(text):
    """Summarize text using Hugging Face's pipeline."""
    try:
        summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
        summary = summarizer(text, max_length=130, min_length=30, do_sample=False)
        return summary[0]["summary_text"]
    except Exception as e:
        return f"Error: {e}"

# Math Visualizations
def generate_math_latex(prompt):
    """Generate LaTeX-based math equations."""
    try:
        response = openai_client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": f"Write LaTeX code for: {prompt}"}],
            temperature=0.2,
            max_tokens=300
        )
        latex_code = response.choices[0].message.content.strip()
        return f"$$ {latex_code} $$"
    except Exception as e:
        return f"Error: {e}"

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# 🚀 **Comprehensive Data and AI Platform**")
    gr.Markdown("Analyze data, generate AI insights, and create math visualizations.")

    # CSV Analysis
    with gr.Tab("CSV Analysis"):
        gr.Markdown("### **Upload Your Data for Comprehensive EDA**")
        csv_file = gr.File(label="Upload CSV File", file_types=[".csv"])
        summary_html = gr.HTML(label="Statistical Summary")
        heatmap = gr.Image(label="Correlation Heatmap")
        histograms = gr.Image(label="Numeric Histograms")
        boxplots = gr.Image(label="Boxplots")
        status = gr.Textbox(label="Status")

        analyze_btn = gr.Button("Analyze CSV")
        analyze_btn.click(
            comprehensive_eda,
            inputs=csv_file,
            outputs=[summary_html, heatmap, histograms, boxplots, status]
        )

    # Text Generation
    with gr.Tab("Text Generation"):
        gr.Markdown("### **Generate High-Quality Text**")
        prompt_input = gr.Textbox(label="Enter a Prompt", placeholder="E.g., Write a research proposal for AI in healthcare.")
        creativity = gr.Slider(label="Creativity Level", minimum=0.0, maximum=1.0, value=0.7)
        max_tokens = gr.Slider(label="Max Tokens", minimum=100, maximum=2000, value=1500, step=100)
        gpt_output = gr.Textbox(label="Generated Text", lines=15)

        gpt_button = gr.Button("Generate Text")
        gpt_button.click(
            gpt_generate_text,
            inputs=[prompt_input, creativity, max_tokens],
            outputs=gpt_output
        )

    # Text Summarization
    with gr.Tab("Text Summarization"):
        gr.Markdown("### **Summarize Long Texts**")
        text_input = gr.Textbox(label="Input Text", placeholder="Paste a research paper or article here.", lines=10)
        summary_output = gr.Textbox(label="Summary", lines=5)

        summarize_btn = gr.Button("Summarize")
        summarize_btn.click(
            summarize_text,
            inputs=text_input,
            outputs=summary_output
        )

    # Math Visualizations
    with gr.Tab("Math Visualizations"):
        gr.Markdown("### **Create Math Visualizations**")
        math_prompt = gr.Textbox(label="Math Prompt", placeholder="E.g., Derive the quadratic formula.")
        math_output = gr.HTML(label="LaTeX Output")

        math_btn = gr.Button("Generate Math Visualization")
        math_btn.click(
            generate_math_latex,
            inputs=math_prompt,
            outputs=math_output
        )

# Launch the App
demo.launch(debug=True, share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a567418c9dbd4cc956.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a567418c9dbd4cc956.gradio.live
